<div style="font-family:Arial Narrow" align="center">
<h1 style="font-size: 50pt">Job Market Analysis</h1> 
<h3 style="font-size: 30pt">An overview of Lombardia job market</h3>
<img src="./src/img1.png" />

</div>

<div style="font-family:Arial Narrow">
    <h1 style="font-size: 40pt">Data</h1>
    <p></p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Activated Contracts from Regione Lombardia</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Ceased Contracts from Regione Lombardia</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;ATECO Code</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;GeoJSON Lombardia</p>
    <img width="200px" height="200px" src="./src/img2.png" align="right" />
</div>

In [136]:
# Retriving all datasets from online sources if we don't have them
import os
import requests

path = "./datasets/"

datasets = [
    {# Activated Contracts Dataset
        'url' : 'https://dati.lombardia.it/api/views/qbau-cyuc/rows.csv?accessType=DOWNLOAD',
        'filename' : 'Rapporti_di_lavoro_attivati.csv'
    },
    {# Ceased Contracts Dataset
        'url' : 'https://www.dati.lombardia.it/api/views/nwz3-p6vm/rows.csv?accessType=DOWNLOAD',
        'filename' : 'Rapporti_di_lavoro_cessati.csv'
    },
    {# ATECO Code Dataset
        'url' : 'https://www.istat.it/it/files//2022/03/Struttura-ATECO-2007-aggiornamento-2022.xlsx',
        'filename' : 'Struttura-ATECO-2007-aggiornamento-2022.xlsx'
    },
    {# GeoJSON border coordinates of Italy provinces from github
        'url' : 'https://dati.lombardia.it/api/views/qbau-cyuc/rows.csv?accessType=DOWNLOAD',
        'filename' : 'limits_IT_provinces.geojson'
    },   
]

# Create datasets dir
if not os.path.exists(path):
    os.mkdir(path)

# Retrive datasets
for dataset in datasets:
    if not os.path.exists(path + dataset["filename"]):
        r = requests.get(dataset["url"], allow_redirects=True)
        open(path + dataset["filename"], 'wb').write(r.content)


<div style="font-family:Arial Narrow">
    <h1 style="font-size: 40pt">Data Cleaning</h1>
    <p></p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Load Datasets</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Remove Outliers</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Join with ATECO Code Datasets</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Manage null values</p>
    <img width="200px" height="200px" src="./src/img3.png" align="right" />
</div>

In [149]:
import pandas as pd
import plotly.express as px
import json

# Ignore warnings for presentation
import warnings
warnings.filterwarnings("ignore")

###############################################################################################################################

def load_datasets():
    df_lav_att = pd.read_csv("./datasets/Rapporti_di_lavoro_attivati.csv")
    df_lav_ces = pd.read_csv("./datasets/Rapporti_di_lavoro_cessati.csv")
    df_ateco = pd.read_excel("./datasets/Struttura-ATECO-2007-aggiornamento-2022.xlsx", engine="openpyxl")
    geo = json.load(open("./datasets/limits_IT_provinces.geojson"))
    
    return df_lav_att, df_lav_ces, df_ateco, geo

###############################################################################################################################

def plot_age_distribution(df):
    # Prepare Data
    df_age = df.groupby(['ETA'])['ETA'].count().to_frame()
    df_age.columns.values[0] = "COUNT"
    df_age.reset_index(inplace = True)
    df_age.columns.values[0] = "AGE"
    df_age.sort_values(['AGE'], inplace=True)

    # Bar plot - Age Distribution
    fig = px.bar(df_age, x="AGE", y="COUNT", title="Age Distribution", width=850, height=300)
    fig.show()

    # Box Plot - Age Distribution
    fig = px.box(df_age, x="AGE", width=850, height=300, points="all")
    fig.show()
    
###############################################################################################################################    
    
def plot_time_distribution(df, title):
   # Trasform date into MM-YYYY
    df['DATA'] = pd.to_datetime(df['DATA'], format="%d/%m/%Y", errors='coerce')
    df.dropna(subset=["DATA"], inplace=True)
    df['DATA'] = df['DATA'].apply(lambda x: x.strftime('%Y-%m'))

    # Prepare Data
    df_time = df.groupby(['DATA'])['DATA'].count().to_frame()
    df_time.columns.values[0] = "COUNT"
    df_time.reset_index(inplace = True)
    df_time.columns.values[0] = "DATE"

    # Scatter plot - Time Series Distribution
    fig = px.line(df_time, x='DATE', y="COUNT", title=title, width=850, height=300)
    fig.show()

###############################################################################################################################    
    
def clean_ateco_dataset(df_ateco):
    # Rename some columns
    df_ateco.columns.values[0] = "CodAteco"
    df_ateco.columns.values[1] = "DescrizioneAteco"

    # Create new features
    df_ateco.insert(2, "MacroAteco", "")
    df_ateco.insert(3, "MacroDescrizione", "")

    df_ateco["MacroAteco"] = df_ateco.apply(lambda x: x["CodAteco"] if x["CodAteco"].isalpha() else "", axis=1)
    df_ateco["MacroDescrizione"] = df_ateco.apply(lambda x: x["DescrizioneAteco"] if x["CodAteco"].isalpha() else "", axis=1)

    macro_cod = ""
    macro_desc = ""
    for x in df_ateco.index:
        if df_ateco["MacroAteco"][x] == "":
            df_ateco["MacroAteco"][x] = macro_cod
            df_ateco["MacroDescrizione"][x] = macro_desc
        else:
            macro_cod = df_ateco["MacroAteco"][x]
            macro_desc = df_ateco["MacroDescrizione"][x]

    # Delete unwanted duplicates (e.g. CodAteco 90.1 and 90.1.0, are the same)
    df_ateco.drop_duplicates(subset=['DescrizioneAteco'], inplace=True)
    return df_ateco
    
def join_with_ateco(df, df_ateco):
    df_ateco.rename(columns={"DescrizioneAteco": "SETTOREECONOMICODETTAGLIO"}, inplace=True)
    df = pd.merge(df, df_ateco, how="left")

    
###############################################################################################################################    
    
def plot_ateco_live(df):
    df["DATA"] = pd.to_datetime(df["DATA"], format="%Y-%m", errors='coerce')
    df["DATA"] = df["DATA"].dt.year
    df = df.groupby(['MacroAteco', 'DATA'])['MacroAteco'].count().to_frame()
    df.columns.values[0] = "COUNT"
    df.reset_index(inplace=True)

    fig = px.line_polar(df, r='COUNT', theta='MacroAteco', animation_frame="DATA", line_close=True)
    fig.update_traces(fill='toself')
    fig.show()

###############################################################################################################################




In [161]:
df_lav_att, df_lav_ces, df_ateco, df_geojson = load_datasets()

In [151]:
plot_age_distribution(df_lav_att)

In [152]:
plot_age_distribution(df_lav_ces)

In [153]:
plot_time_distribution(df_lav_att, "Activated Contracts")
plot_time_distribution(df_lav_ces, "Ceased Contracts")

In [162]:
df_ateco = clean_ateco_dataset(df_ateco)
df_lav_att = pd.merge(df_lav_att, df_ateco, how="left", left_on="SETTOREECONOMICODETTAGLIO", right_on="DescrizioneAteco")
df_lav_ces = pd.merge(df_lav_ces, df_ateco, how="left", left_on="SETTOREECONOMICODETTAGLIO", right_on="DescrizioneAteco")

In [166]:
plot_ateco_live(df_lav_att)

<ipython-input-149-3ed77e086591>:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-149-3ed77e086591>:88: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [165]:
#df_lav_att['DATA'] = pd.to_datetime(df_lav_att['DATA'], format="%d/%m/%Y", errors='coerce')
from_date = "2009-01-01"
to_date = "2021-12-31"
df_lav_att = df_lav_att.loc[(df_lav_att["DATA"]>=from_date) & (df_lav_att["DATA"]<=to_date)]
df_lav_att

,DATA,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,CodAteco,DescrizioneAteco,MacroAteco,MacroDescrizione
0,2020-05-09,F,60,Attività di famiglie e convivenze come datori ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,97.00,Attività di famiglie e convivenze come datori ...,T,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,2019-07-12,M,43,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,NaN,NaN,NaN,NaN
2,2013-06-05,F,20,Fabbricazione di altre apparecchiature elettri...,LICENZA MEDIA,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA,NaN,NaN,NaN,NaN
3,2010-03-12,F,28,Alberghi,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERMITTENTE A TEMPO DETERMINATO,NON DEFINITO,BERGAMO,ITALIA,55.10.0,Alberghi,I,ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZ...
4,2021-04-06,F,49,Rifugi di montagna,LICENZA MEDIA,LAVORO INTERMITTENTE,NON DEFINITO,BERGAMO,ITALIA,55.20.3,Rifugi di montagna,I,ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9579789,2020-03-01,M,52,Lavori di meccanica generale,LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,25.62,Lavori di meccanica generale,C,ATTIVITÀ MANIFATTURIERE
9579790,2010-09-06,M,61,Lavori di meccanica generale,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,BERGAMO,ALBANIA,25.62,Lavori di meccanica generale,C,ATTIVITÀ MANIFATTURIERE
9579791,2021-11-06,M,37,Fabbricazione di parti ed accessori per bicicl...,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,SENEGAL,30.92.2,Fabbricazione di parti ed accessori per bicicl...,C,ATTIVITÀ MANIFATTURIERE
9579792,2010-02-02,M,35,Fabbricazione di parti ed accessori per bicicl...,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,TEMPO PIENO,BERGAMO,SENEGAL,30.92.2,Fabbricazione di parti ed accessori per bicicl...,C,ATTIVITÀ MANIFATTURIERE


In [160]:
# Remove record with Date < 01/2009 and Date > 12/2021
from_date = "2009-01-01"
to_date = "2021-12-31"
df_lav_att['DATA'] = pd.to_datetime(df_lav_att['DATA'], format="%d/%m/%Y", errors='coerce')
df_lav_ces['DATA'] = pd.to_datetime(df_lav_ces['DATA'], format="%d/%m/%Y", errors='coerce')
df_lav_att = df_lav_att.loc[(df_lav_att["DATA"]>=from_date) & (df_lav_att["DATA"]<=to_date)]
df_lav_ces = df_lav_ces.loc[(df_lav_ces["DATA"]>=from_date) & (df_lav_ces["DATA"]<=to_date)]
df_lav_att


,DATA,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,CodAteco,DescrizioneAteco,MacroAteco,MacroDescrizione


<h1 style="font-family:Arial Narrow">Data Exploration</h1> 

In [ ]:
# Data Exploration

<h1 style="font-family:Arial Narrow">Prediction Models</h1> 

In [ ]:
# Prediction Models

<h1 style="font-family:Arial Narrow">Prediction Results</h1> 

In [ ]:
# Prediction Results

<h1 style="font-family:Arial Narrow">Conclusions</h1> 

<h1 style="font-family:Arial Narrow">Improvements</h1> 

<h1 style="font-family:Arial Narrow" align="center">Thanks for your attention!</h1>